In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.linalg import toeplitz
from statsmodels.tsa.arima.model import ARIMA
from itertools import product

In [6]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm

# Define the date range for historical data
start_date = '2018-01-01'
end_date = '2024-06-30'

# Download stock and market index prices (AAPL & SPY as market proxy)
data = yf.download(['AAPL', 'SPY'], start=start_date, end=end_date, progress=False)['Close']

# Compute daily percentage returns
returns = data.pct_change().dropna()
returns.columns = ['AAPL_Return', 'Market_Return']

# URL for Fama-French factors dataset
ff_url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip"

# Load Fama-French data
ff_data = pd.read_csv(ff_url, skiprows=3)  # Skip metadata rows

# Rename first column to 'Date' and filter numeric values only
ff_data = ff_data.rename(columns={ff_data.columns[0]: "Date"})
ff_data = ff_data[ff_data['Date'].str.isnumeric()]  # Remove non-date rows

# Convert 'Date' column to datetime format
ff_data['Date'] = pd.to_datetime(ff_data['Date'], format='%Y%m%d')

# Set 'Date' as the index and scale percentage values to decimal format
ff_data = ff_data.set_index('Date')
ff_data = ff_data.astype(float) / 100  # Convert from percentage format

# Align Fama-French data with stock returns time range
start_date = returns.index.min()
end_date = returns.index.max()
ff_data = ff_data.loc[start_date:end_date]  

# Merge stock returns with Fama-French factors (Market-RF, SMB, HML, RF)
returns = returns.merge(ff_data[['Mkt-RF', 'SMB', 'HML', 'RF']], left_index=True, right_index=True)

# Compute excess returns for AAPL and the market
returns['Excess_AAPL'] = returns['AAPL_Return'] - returns['RF']
returns['Excess_Market'] = returns['Market_Return'] - returns['RF']

# Define independent variables (Fama-French three factors)
X = returns[['Mkt-RF', 'SMB', 'HML']]
X = sm.add_constant(X)  # Add intercept term

# Define dependent variable (excess return of AAPL)
y = returns['Excess_AAPL']

# Fit the Fama-French three-factor model using OLS regression
ff_model = sm.OLS(y, X).fit()

# Print regression summary
print(ff_model.summary())

                            OLS Regression Results                            
Dep. Variable:            Excess_AAPL   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.660
Method:                 Least Squares   F-statistic:                     1055.
Date:                Sat, 15 Feb 2025   Prob (F-statistic):               0.00
Time:                        12:46:22   Log-Likelihood:                 4979.0
No. Observations:                1632   AIC:                            -9950.
Df Residuals:                    1628   BIC:                            -9928.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.000      1.652      0.0

In [7]:
import warnings
import numpy as np
import pandas as pd
from itertools import product
from statsmodels.tsa.arima.model import ARIMA

# Extract residuals from the Fama-French regression model
residuals = pd.Series(ff_model.resid)

# Function to find the best ARIMA model based on AIC (Akaike Information Criterion)
def find_best_arima(data, p_range, d_range, q_range):
    best_aic = np.inf  # Initialize AIC with a large value
    best_order = None  # Store the best (p, d, q) order
    best_model = None  # Store the best ARIMA model

    # Iterate through all possible combinations of (p, d, q)
    for order in product(p_range, d_range, q_range):
        try:
            # Fit ARIMA model with current order
            model = ARIMA(data, order=order).fit()

            # Update best model if the current AIC is lower than the best found so far
            if model.aic < best_aic:
                best_aic = model.aic
                best_order = order
                best_model = model
        except:
            continue  # Skip configurations that cause errors

    return best_model, best_order  # Return the best model and its order

# Define parameter search space for ARIMA (p, d, q)
p_range = range(0, 4)  # Autoregressive terms
d_range = range(0, 2)  # Differencing terms
q_range = range(0, 4)  # Moving average terms

# Suppress ARIMA-related warnings for cleaner output
warnings.filterwarnings("ignore", message=".*A date index has been provided.*")
warnings.filterwarnings("ignore", category=UserWarning, message=".*Non-stationary starting autoregressive parameters.*")
warnings.filterwarnings("ignore", category=UserWarning, message=".*Non-invertible starting MA parameters found.*")
warnings.filterwarnings("ignore", category=UserWarning, message=".*Maximum Likelihood optimization failed to converge.*")

# Find the best ARIMA model for residuals
model, best_order = find_best_arima(residuals, p_range, d_range, q_range)

# Print the best ARIMA model order
print(f"Best ARIMA Order for Residuals: {best_order}")


Best ARIMA Order for Residuals: (0, 0, 1)


In [8]:
import numpy as np
from scipy.linalg import toeplitz

# Extract variance (sigma²) from the fitted ARIMA model
sigma2 = model.scale 

# Extract AR and MA parameters from the model (handling cases with no parameters)
ar_params = model.arparams if model.arparams.size > 0 else np.array([])
ma_params = model.maparams if model.maparams.size > 0 else np.array([])

# Construct AR and MA polynomials (include leading 1 for stability)
ar_poly = np.r_[1, -ar_params] if ar_params.size else np.array([1])
ma_poly = np.r_[1, ma_params] if ma_params.size else np.array([1])

# Function to compute the autocovariance function (ACF) manually
def compute_acovf(ar, ma, sigma2, lags):
    """
    Compute the autocovariance function up to a given number of lags.

    Parameters:
    - ar: AR polynomial coefficients
    - ma: MA polynomial coefficients
    - sigma2: Residual variance (sigma²)
    - lags: Number of lags to compute

    Returns:
    - acovf: Autocovariance values up to 'lags'
    """
    acovf = np.zeros(lags)
    
    # Compute lag-0 variance (based on AR and MA terms)
    acovf[0] = sigma2 * (1 + np.sum(ma**2)) / (1 - np.sum(ar**2)) if np.sum(ar**2) < 1 else sigma2

    # Compute higher-lag autocovariances using recursion
    for lag in range(1, lags):
        ar_slice = ar[:min(lag, len(ar))]  # Select AR terms up to current lag
        acovf_slice = acovf[:lag][::-1]  # Reverse previous ACF values

        # Handle different array lengths by padding with zeros
        if len(ar_slice) != len(acovf_slice):
            diff = abs(len(ar_slice) - len(acovf_slice))
            if len(ar_slice) < len(acovf_slice):
                ar_slice = np.pad(ar_slice, (0, diff))
            else:
                acovf_slice = np.pad(acovf_slice, (0, diff))

        # Compute autocovariance using dot product
        acovf[lag] = np.dot(ar_slice, acovf_slice)

    return acovf

# Compute autocovariances up to the length of residuals
acovf = compute_acovf(ar_params, ma_params, sigma2, len(residuals))

# Construct the covariance matrix using the Toeplitz structure
cov_matrix = toeplitz(acovf)


In [9]:
#GLS

ff_gls = sm.GLS(y, X, sigma=cov_matrix).fit()
print(ff_gls.summary())

                            GLS Regression Results                            
Dep. Variable:            Excess_AAPL   R-squared:                       0.660
Model:                            GLS   Adj. R-squared:                  0.660
Method:                 Least Squares   F-statistic:                     1055.
Date:                Sat, 15 Feb 2025   Prob (F-statistic):               0.00
Time:                        12:46:43   Log-Likelihood:                 4979.0
No. Observations:                1632   AIC:                            -9950.
Df Residuals:                    1628   BIC:                            -9928.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.000      1.652      0.0